# Landfire Data Ingest Tutorial

In this project, elevation above sea level is utilized as a predictor of FMC. RAWS stations report a elevation that is used when constructing training data.

For forecasting to arbitrary locations, we need elevation data. The HRRR weather model utilizes a geopotential height in it's modeling, but this may not be directly comparable to the elevation as reported by a RAWS station. In this project, we will utilize elevation data from [LandFire](https://landfire.gov/topographic/elevation), which utilizes 3D mapping from USGS. The elevation data is very high resolution; it has approximately 30m resolution and is contained in a file that is over 8gb. 

In this project we utilize the LandFire elevation data as follows:
* Download the data for all of CONUS (this takes while)
* From the LandFire elevation data, we get the elevation that corresponds to each HRRR grid node and save it locally. A file of these elevations is much smaller, since HRRR operates on a 3km grid.
* This smaller file of elevation data that corresponds to the HRRR grid is saved locally, and it is also staged for download on the OpenWFM project site DEMO (TODO)

Due to computational limitations, the operations were performed with gdal on the command line in Alderaan:


This notebook demonstrates the process of downloading the LandFire elevation data and extracting points that correspond to the HRRR grid. This is a slow process that should only be run on project setup once. Additionally, the elevation data at HRRR grid nodes can be downloaded online, thus avoiding this time consuming process, but the code is included for reproducibility.

*NOTE*: LandFire data too big for processing in memory in a jupyter notebook. TODO: use gdal command line. Reference: https://nsidc.org/data/user-resources/help-center/how-do-i-reproject-geotiff-polar-stereographic-geographic-latlon

Canopy Cover (CC): https://landfire.gov/fuel/cc

## Setup

In [ ]:
import numpy as np
import zipfile
import sys
import os.path as osp
import rioxarray as rxr
import matplotlib.pyplot as plt
import herbie
sys.path.append("../src")
from utils import retrieve_url

In [ ]:
# Determine Paths and Filenames
data_path = "../data"

# Elevation data
elev_url = "https://landfire.gov/data-downloads/US_Topo_2020/LF2020_Elev_220_CONUS.zip"
elev_zipname = "LF2020_Elev_220_CONUS.zip"
elev_path = osp.join(data_path, "LF2020_Elev_220_CONUS", "Tif", "LC20_Elev_220.tif")

# Canopy Cover (CC) Data
canopy_url = "https://landfire.gov/data-downloads/US_230/LF2022_CC_230_CONUS.zip"
canopy_zipname = "LF2022_CC_230_CONUS.zip"
canopy_path = osp.join(data_path, "LF2022_CC_230_CONUS", "Tif", "LC22_CC_230.tif")

## Retrieve

* Get the LandFire zip file and save locally, then unzip.
* Get one HRRR grib file to extract the projection info. Use `Herbie` to get the file, save, and then read with `rioxarray`

In [ ]:
# Retrieve Elevation
retrieve_url(
    url = elev_url,
    dest_path = osp.join(data_path, elev_zipname)
)

if not osp.exists(elev_path):
    print("Unzipping file")
    with zipfile.ZipFile(osp.join(data_path, elev_zipname), 'r') as zip_ref:
        zip_ref.extractall(data_path)    

In [ ]:
# Retrieve Canopy
retrieve_url(
    url = canopy_url,
    dest_path = osp.join(data_path, canopy_zipname)
)

if not osp.exists(canopy_path):
    print("Unzipping file")
    with zipfile.ZipFile(osp.join(data_path, canopy_zipname), 'r') as zip_ref:
        zip_ref.extractall(data_path)  

## Read and Visualize

In [ ]:
lf = rxr.open_rasterio("../data/elev_reproject.tif")

In [ ]:
lf = lf.where(lf != -9999)

In [ ]:
lf.shape

In [ ]:
plt.imshow(lf.values[0, :, :])
plt.scatter(100, 0, color='red', s=50, edgecolor='black')

In [ ]:
H = herbie.Herbie("2025-01-01", product="prs")
ds = H.xarray("(?:HGT|LAND):surface")
ds

In [ ]:
ds["orog"] = ds.orog.where(ds.lsm > 0)

In [ ]:
ds.orog.shape

In [ ]:
ds['elev'] = lf.squeeze().sel(y=slice(None, None, -1))

In [ ]:
plt.imshow(ds.orog, origin="lower")

In [ ]:
plt.imshow(ds.elev, origin="lower")

In [ ]:
ds["elev"] = ds.elev.where(ds.lsm > 0)

In [ ]:
lf_missing = np.isnan(ds.elev) & ~np.isnan(ds.orog)
yind, xind = np.where(lf_missing)

In [ ]:
plt.imshow(ds.elev, origin="lower")
plt.scatter(xind, yind, color="red", s=5)

In [ ]:
elev_valid = ds.elev.where(~lf_missing)
orog_valid = ds.orog.where(~lf_missing)
difference = elev_valid - orog_valid

In [ ]:
np.mean(difference)

In [ ]:
np.min(difference)

In [ ]:
np.max(difference)

In [ ]:
# Coordinates for the scatter point
min_diff_value = difference.min().values
min_diff_coords = np.unravel_index(np.nanargmin(difference.values), difference.shape)

# Coordinates for the scatter point
y_min, x_min = min_diff_coords

# Coordinates for the scatter point
max_diff_value = difference.max().values
max_diff_coords = np.unravel_index(np.nanargmax(difference.values), difference.shape)

# Coordinates for the scatter point
y_max, x_max = max_diff_coords

In [ ]:
plt.imshow(ds.elev, origin="lower")
plt.scatter(xind, yind, color="red", s=5, label=f"Missing in LF")
plt.scatter(x_min, y_min, color="blue", s=50, label=f"Min Difference: {min_diff_value:.2f}")
plt.scatter(x_max, y_max, color="black", s=50, label=f"Max Difference: {max_diff_value:.2f}")